# Sentiment Analysis
## Use *TextBlob*

Generate a sentiment analysis for newspaper articles covering events in Syria from the years 2010-2017. 

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from numpy import nan
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os

sns.set_context('notebook')
sns.set_style('whitegrid')

## Data Loading

In [3]:
df = pd.read_csv('CleanLexisNexis.csv', parse_dates=['date'])

In [4]:
df.dtypes

publication                object
date               datetime64[ns]
title                      object
length                      int64
publicationtype            object
text                       object
year                        int64
month                       int64
day                         int64
dtype: object

In [5]:
df.head(4)

,publication,date,title,length,publicationtype,text,year,month,day
0,The Atlanta Journal-Constitution,2010-01-03,Five pressing questions to answer in 2010,747,Newspapers,Will President Barack Obama regain his momentu...,2010,1,3
1,BBC,2010-01-04,"Saudi foreign minister says Israel ""spoiled ch...",2196,Transcript,Text of report by Saudi-owned leading pan-Arab...,2010,1,4
2,BBC,2010-01-08,Highlights of Iran parliamentary session.,1123,Transcript,Excerpt from report on parliamentary proceedin...,2010,1,8
3,Right Vision News,2010-01-09,Jordan:Way out for Obama,852,Newspaper,"Pakistan, Jan. 09 -- These are the worst of ti...",2010,1,9


## 1. Sentiment Analysis

Use NLTK to build sentiment scores. 

Use the positive/negative corpus provided by Andy Kim, author of *Can Big Data Forcast North Korean Military Aggression?* 

#### Append positive and negative list together

In [7]:
os.chdir('/Users/laurieottehenning/Documents/Georgetown Data Science /Capstone/Harvard Pos:Neg')

pos = pd.read_csv('Harvard_Positive.csv', names=['Word', 'Positive'])
neg = pd.read_csv('Harvard_Negative.csv', names=['Word', 'Negative'])

pos['Word'] = pos['Word'].str.lower()
neg['Word'] = neg['Word'].str.lower()

wordlist = pd.concat([pos, neg])


#### Count the number of positive or negative words within a text

In [11]:
pos_list = []
for i in pos['Word']:
    pos_list.append(i)
 

neg_list = []
for i in neg['Word']:
    neg_list.append(i)

df = df.assign(PositiveCount=df['text'].apply(lambda sentence: 
                                            sum(word.lower() in pos_list 
                                                for word in sentence.split())))
df = df.assign(NegativeCount=df['text'].apply(lambda sentence: 
                                            sum(word.lower() in neg_list 
                                                for word in sentence.split())))



In [12]:
df.head(4)

,publication,date,title,length,publicationtype,text,year,month,day,PositiveCount,NegativeCount
0,The Atlanta Journal-Constitution,2010-01-03,Five pressing questions to answer in 2010,747,Newspapers,Will President Barack Obama regain his momentu...,2010,1,3,39,51
1,BBC,2010-01-04,"Saudi foreign minister says Israel ""spoiled ch...",2196,Transcript,Text of report by Saudi-owned leading pan-Arab...,2010,1,4,137,89
2,BBC,2010-01-08,Highlights of Iran parliamentary session.,1123,Transcript,Excerpt from report on parliamentary proceedin...,2010,1,8,50,38
3,Right Vision News,2010-01-09,Jordan:Way out for Obama,852,Newspaper,"Pakistan, Jan. 09 -- These are the worst of ti...",2010,1,9,51,38


#### Use SentiWordNet

In [7]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
 
 
lemmatizer = WordNetLemmatizer()
 
 
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
 
def clean_text(text):
    text = text.replace("<br />", " ")
    text = text.decode("utf-8")
 
    return text
 
 
def swn_polarity(text):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """
 
    sentiment = 0.0
    tokens_count = 0
 
    text = clean_text(text)
 
 
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
 
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
 
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
 
    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0
 
    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1
 
    # negative sentiment
    return 0

#### Create Classifier to be trained on Newspaper data

In [30]:
with open('wordlist.csv') as fp:
    cl = NaiveBayesClassifier(fp, format="csv")

In [39]:
# blob = df['text'].apply(lambda tweet: TextBlob(tweet, classifier=cl)

df['polarity'] = df.apply(lambda x: TextBlob(x['text'], classifier=cl).sentiment.polarity, axis=1)
df['subjectivity'] = df.apply(lambda x: TextBlob(x['text']).sentiment.subjectivity, axis=1)

In [40]:
df.ix[2000:2010]

,publication,date,title,length,publicationtype,text,year,month,day,polarity,subjectivity
2000,The Christian Science Monitor,2012-07-23,Obama vs. Romney: VFW hosting campaign side tr...,779,Newspaper,President Obama told the Veterans of Foreign W...,2012,7,23,0.047170,0.302023
2001,The Times (London),2012-07-24,Romney will be courted by Labour as he forges ...,613,Newspaper,Ed Miliband will try to find common cause with...,2012,7,24,0.098021,0.384447
2002,The Times (London),2012-07-24,Romney will be courted by Labour as he forges ...,615,Newspaper,Ed Miliband will try to find common cause with...,2012,7,24,0.098021,0.384447
2003,"The Age (Melbourne, Australia)",2012-07-24,America's foreign fantasy,970,Newspaper,Washington's global ambitions exceed its power...,2012,7,24,0.060325,0.386745
2004,BBC,2012-07-24,Iran MP says foreign interference in Syria to ...,470,Transcript,Text of report on interview with Javad Jahangi...,2012,7,24,0.060412,0.327679
2005,BBC,2012-07-24,Programme summary of Afghan Tolo TV news in Da...,621,Transcript,A. News Headlines B. Home News 1. 0030 A repor...,2012,7,24,-0.031420,0.265370
2006,Brattleboro Reformer (Vermont),2012-07-25,World in Brief,1347,Newspaper,"Wednesday July 25, 2012 Nonpartisan budget off...",2012,7,25,0.097203,0.410523
2007,Scotsman,2012-07-25,Tavish Scott: This diplomatic stand-off result...,464,Newspaper,The greatest show on Earth is about to begin. ...,2012,7,25,0.147222,0.331574
2008,The Christian Science Monitor,2012-07-25,What would 'President Romney' do about Syria?;...,951,Newspaper,"Judging from headlines, one might think Mitt R...",2012,7,25,0.074726,0.363221
2009,Charleston Gazette (West Virginia),2012-07-25,NATIONAL Briefs,932,Newspaper,"Romney says Obama threat to security RENO, Nev...",2012,7,25,0.089305,0.389729
